In [1]:
import pandas as pd
import numpy as np  
import sys

import PyUber

In [2]:
xeus_source = 'F32_PROD_XEUS'

In [12]:
query = '''
    Select Distinct
        FLF.LOT
        ,FLF.LOT_PROCESS
        ,FLF.PRODUCT
        ,FLF.OPERATION
        ,FLF.OPER_SHORT_DESC
        ,O.oper_long_desc AS OPER_LONG_DESC
        ,O.area AS AREA
        ,O.module AS MODULE
        ,FLF.EXEC_SEQ
        ,FLF.OUT_DATE
        ,FLF.INSTRUCTION_FLAG
        ,FLF.SPLIT_FLAG
        ,FLF.MERGE_FLAG
        ,FLF.SIF_FLAG
    From
        F_LOT_FLOW FLF,
        F_OPERATION O

    WHERE
        LOT = 'A5280130'
        AND O.operation = FLF.operation
        AND O.latest_version = 'Y'
        and FLF.history_deleted_flag = 'N'
    ORDER BY
        FLF.EXEC_SEQ
'''


In [13]:
with PyUber.connect(datasource=xeus_source) as conn:
    df = pd.read_sql(query, conn)

C:\Users\dagarcia\AppData\Local\Temp\ipykernel_14484\3968044218.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [ ]:
df
